In [ ]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("article#div_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:채권

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):2

크롤링할 종료 페이지:  2 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=채권&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=채권&start=11']


0it [00:00, ?it/s]
0it [00:00, ?it/s]

검색된 기사 갯수: 총  20 개

[뉴스 제목]
[]

[뉴스 링크]
[]

[뉴스 내용]
[]
news_title:  0
news_url:  0
news_contents:  0
news_dates:  0
중복 제거 후 행 개수:  0


In [2]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install fake_useragent

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 341 kB in 3s (135 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0

In [3]:
!pip install fake_useragent

In [3]:
import time
import requests

from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from pytz import timezone
from openpyxl import Workbook
from bs4 import BeautifulSoup


wb = Workbook()
ws = wb.active
ws.append(["제목", "내용", "링크"])


ua = UserAgent()
headers = {
    "User-Agent" : ua.random
}

# 크롬 드라이버 설정
options = ChromeOptions()
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
options.add_argument('user-agent=' + user_agent)
options.add_argument("lang=ko_KR")
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("--no-sandbox")

# 크롬 드라이버 최신 버전 설정
service = ChromeService()

# chrome driver
d = webdriver.Chrome(service=service, options=options) # <- options로 변경
keyword = input("키워드를 입력해주세요.")
# print("검색한 키워드 : "+keyword)

news_title = []
news_content = []
news_link = []

try:
    d.get("https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101")
    time.sleep(0.5)

    for i in range(2, 12):
        # 수집
        section = d.find_element(By.CSS_SELECTOR, ".section_body")
        lis = section.find_elements(By.CSS_SELECTOR, "ul > li")

        for li in lis:
            dts = li.find_elements(By.CSS_SELECTOR, "dl > dt")
            link = li.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            title = dts[1].text.replace("'", '"')

            # 링크 안에 있는 내용을 읽어오기
            res = requests.get(link, headers=headers)
            bs = BeautifulSoup(res.text, 'html.parser')
            content = bs.select_one("#dic_area").text

            if keyword in title or keyword in content:
                news_title.append(title)
                news_content.append(content)
                news_link.append(link)

                ws.append([title, content, link])

        print(news_title)

        # 클릭
        page_area = d.find_element(By.CSS_SELECTOR, "#paging")
        page_2 = page_area.find_element(By.LINK_TEXT, str(i)) # 링크된 텍스트를 클릭해서 페이지를 넘어갈수도 있다.
        if i > 1 and i % 10 == 1:
            page_2 = page_area.find_element(By.LINK_TEXT, "다음")
            print(page_2)
        page_2.click()

        time.sleep(1)

except Exception as e:
    print(e)
finally:
    time.sleep(2)
    d.close()
    d.quit()
    wb.save("news.csv")

키워드를 입력해주세요.금리
['[이윤희의 매크로 VIEW] "금리의 보복"', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다']
['[이윤희의 매크로 VIEW] "금리의 보복"', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '3분기 금융지주 실적 전망…KB "양호" vs 신한·하나·우리 "부진"', '컨트리가든 "디폴트" 현실화 하나…구조조정 자문사 선임', '"올 초 8억 하던 아파트가 벌써 12억"…세입자들 "발 동동"', '역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 "속탄다"']
['[이윤희의 매크로 VIEW] "금리의 보복"', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '3분기 금융지주 실적 전망…KB "양호" vs 신한·하나·우리 "부진"', '컨트리가든 "디폴트" 현실화 하나…구조조정 자문사 선임', '"올 초 8억 하던 아파트가 벌써 12억"…세입자들 "발 동동"', '역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 "속탄다"', '전쟁나도 오르는 코스피 "안심은 일러…연말까진 상방보다 "하방" 열려"']
['[이윤희의 매크로 VIEW] "금리의 보복"', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다', '3분기 금융지주 실적 전망…KB "양호" vs 신한·하나·우리 "부진"', '컨트리가든 "디폴트" 현실화 하나…구조조정 자문사 선임', '"올 초 8억 하던 아파트가 벌써 12억"…세입자들 "발 동동"', '역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 "속탄다"', '전쟁나도 오르는 코스피 "안심은 일러…연말까진 상방보다 "하방" 열려"', '"구리 공급량 감소 추이 심각하다"…청정에너지 전환 더뎌지나', '이스라엘-하마스 전쟁에도…장중 환율, 1340원 중후반

In [4]:
!pip install -U pip
!pip install transformers
!pip install datasets
!pip install tensorflow-gpu==2.8.0
!pip install protobuf==3.20

In [14]:
!pip install chardet

In [5]:
import pandas as pd
import chardet

df = pd.read_excel('news.csv')
df['내용'] = df['내용'].str.replace('\s+', ' ')
df

<ipython-input-5-ca3a6cb309fa>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['내용'] = df['내용'].str.replace('\s+', ' ')


,제목,내용,링크
0,"[이윤희의 매크로 VIEW] ""금리의 보복""",연합뉴스 어쩌면 밀린 공과금 청구서나 군자의 복수 계획처럼 유예됐을 뿐 언젠가는 ...,https://n.news.naver.com/mnews/article/029/000...
1,"사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다",게티이미지뱅크일상 속 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 주화가 급증...,https://n.news.naver.com/mnews/article/469/000...
2,"사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다",게티이미지뱅크일상 속 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 주화가 급증...,https://n.news.naver.com/mnews/article/469/000...
3,"3분기 금융지주 실적 전망…KB ""양호"" vs 신한·하나·우리 ""부진""","SK증권 “KB금융, 양호한 대출 성장 바탕으로 안정적 이자이익 기록”하나증권 “...",https://n.news.naver.com/mnews/article/123/000...
4,"컨트리가든 ""디폴트"" 현실화 하나…구조조정 자문사 선임","中집값 급락에 보유자산 처분 막혀…자문사 선정, 17일까지 역외채권 이자 1500...",https://n.news.naver.com/mnews/article/008/000...
5,"""올 초 8억 하던 아파트가 벌써 12억""…세입자들 ""발 동동""","""전세 매물, 다 어디 갔나요""가을 이사철, 세입자들 '발 동동'전셋값 상승·매물...",https://n.news.naver.com/mnews/article/015/000...
6,"역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 ""속탄다""","직방 ""응답자 55%는 '그래도 연내 추가 대출 필요' 답변""고금리 상황이 지속되...",https://n.news.naver.com/mnews/article/374/000...
7,"전쟁나도 오르는 코스피 ""안심은 일러…연말까진 상방보다 ""하방"" 열려""",'확전' 가능성 차단 및 추가 금리인상 일축에 반등고금리 지속으로 투심 악화 당분...,https://n.news.naver.com/mnews/article/421/000...
8,"""구리 공급량 감소 추이 심각하다""…청정에너지 전환 더뎌지나",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...
9,"이스라엘-하마스 전쟁에도…장중 환율, 1340원 중후반대 약보합[외환분석]",중동 전쟁에 안전자산 선호…국고채 10년 13bp 하락달러인덱스 장중 106선 하...,https://n.news.naver.com/mnews/article/018/000...


In [9]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from tqdm import tqdm  # tqdm 라이브러리를 임포트합니다.

#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

# '요약' 컬럼을 생성하는 함수 정의
def generate_summary(content):
    input_ids = tokenizer.encode(content, return_tensors="pt", max_length=1024, truncation=True)
    print(content[:2400])
    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        length_penalty=2.0,
        max_length=300,
        min_length=56,
        num_beams=4,
    )
    summary = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    print(summary)
    return summary

# '내용' 컬럼에 대해 '요약' 컬럼 생성
tqdm.pandas()  # tqdm을 사용하여 진행 상황 표시
df['요약'] = df['내용'].progress_apply(generate_summary)

# 결과 확인
print(df)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
  0%|          | 0/36 [00:00<?, ?it/s]

 연합뉴스 어쩌면 밀린 공과금 청구서나 군자의 복수 계획처럼 유예됐을 뿐 언젠가는 반드시 돌아올 것이었을까요. 글로벌 금융위기와 코로나19 사태를 지나며 세계 각국이 공중에 뿌려댄 돈의 대가는 꽤 무자비합니다. 과장해서 '인플레'로 죽을지 부채에 깔려 죽을지를 선택해야 할 상황에 놓이고 말았으니까요. 그동안에도 목이 빠져라 금리 인하 소식을 기다리며 울다가 웃기를 반복하던 시장도 최근엔 각성한 눈치입니다. '더 높은 기준금리를 더 오래'(higher for longer) 유지하겠다는 미국 연방준비제도(Fed·연준)의 메세지에 뒤늦게 반응하며 미국채 시장이 급등했거든요. 이달 초 미국채 10년 만기 국채 수익률이 5%를 육박하며 2007년 이후 최고치를 기록했습니다. 시장으로 풀려난 돈은 증시와 부동산 시장으로 가서 자산 가격을 부풀리고, 창업 시장으로 가 만년 적자 스타트업의 가치를 눈덩이처럼 키웠습니다. 사실 저금리의 달콤함도 누구나 누릴 수 있는 것은 아니었죠. 서민들에게 있어 금리가 낮은 대출은 그저 일시적인 위약 효과에 불과했습니다. 금리가 제아무리 낮은들 저금리로 대출받을 수 있는 고신용자나 담보 보유자들은 정해져 있고, 결국 부채의 만기는 돌아오고야 말테니까요.국경을 막아 선 역사적인 전염병과 예상치 못한 러시아-우크라이나 전쟁에 따른 수급 불안까지 겹치면서 인플레이션은 마치 예정된 수순처럼 우리를 찾아왔습니다. 인플레이션 또한 경제 취약층에 더욱 고통스러운 것입니다. 미국 중앙은행 연준도 기준금리를 인상할 수 밖에 없었습니다. 하지만 좀 늦었던 것 같습니다. 제롬 파월 연준 의장은 폴 볼커만큼 단호하지도, 아서 번즈보다 빠르지도, 앨런 그린스펀처럼 약지도 못했습니다. 코로나19가 한창이던 지난 2021년 파월은 인플레이션은 '일시적(transitory)'이라고 오판했던 것입니다. 그는 근원물가 오름세보다 더디게 금리를 올리기 시작했고, 공격적인 금리인상에도 물가는 잡힐듯 잡히지 않았죠. 11차례의 공격적 금리인상 끝에 미국 소비자물가지수(CPI)와

  6%|▌         | 2/36 [00:07<02:03,  3.63s/it]

글로벌 금융위기와 코로나19 사태를 지나며 세계 각국이 공중에 뿌려댄 돈의 대가는 무자비하며 과장해서 '인플레'로 죽을지 부채에 깔려 죽을지를 선택해야 할 상황에 놓이고 말았으니 원달러 환율은 1400원대를 향하고 국내 시장에서 자본 이탈이 가속화할 수 있다는 겁니다.
 게티이미지뱅크일상 속 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 주화가 급증하고 있는 것으로 나타났다.10일 국회 기획재정위원회 소속 서영교 더불어민주당 의원이 한국은행에서 받은 자료에 따르면, 올해 들어 7월까지 주화 순환수 금액(환수액-발행액)은 156억 원으로 집계됐다. 지난해 같은 기간(67억 원)의 두 배를 웃도는 규모다.시중 수요가 감소함에 따라 주화 재발행 속도를 늦춘 결과로 풀이된다. 한은은 2018년까지만 해도 주화 수요가 높았지만, 카드 등 현금이 아닌 지급 수단 이용이 확대되면서 주화 사용이 크게 줄고 한은 금고로의 환수가 급증하고 있다고 설명했다.잠자는 동전의 재유통을 촉진하기 위해 2008년부터 해마다 실시한 ‘범국민 동전교환운동’은 코로나19 확산을 기점으로 중단된 상태다. 이 운동으로 2008년부터 2018년까지 총 31억 개(4,154억 원)의 동전이 회수됐다. 2019년 5월에도 2억2,100만 개(322억 원)의 동전이 은행권으로 교환됐는데, 이것이 마지막이었다.주화 순환수액 급증에 대해 서 의원은 “잠자는 주화를 재유통하는 경제적 측면, 필요한 곳에서 주화가 적절히 사용되도록 하는 자원 배분 효율성 등이 계속 고려돼야 한다”고 말했다.반면 장롱 속 지폐는 다시 밖으로 나오는 추세다. 올해 1~7월 화폐 환수율은 85%로 코로나19 이전인 2019년의 71.3%를 넘어섰다. 특히 한때 20% 초반대까지 떨어졌던 5만 원권 환수율이 올 7월까지 76.3%에 달해 역대 최고 수준을 기록했다. 예금 금리가 오르면서 화폐 보유의 기회비용이 커지고, 거리두기 해제로 대면 상거래가 정상화한 영향이라고 한은은 분석했다. 


  8%|▊         | 3/36 [00:14<02:55,  5.31s/it]

...

10일 한국은행이 국회 기획재정위원회 소속 서영교 더불어민주당 의원에 제출한 자료에 따르면 올해 7월까지 주화 순환수 금액은 156억원으로, 지난해 같은 기간 67억원의 2배 이상으로 늘고 있지만 지폐 사용은 최근 증가세를 나타내는 등, 올해 시중의 카드 및 간편결제 이용이 보편화되고 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 동전이 급증하고 있다.

 고금리 여파로 빚을 제때 갚지못한 2금융권 차주가 늘어나면서 저축은행권이 한국자산관리공사(캠코)로 넘긴 부실채권 규모가 급증한 것으로 나타났다.10일 국회 정무위원회 소속 양정숙 의원실이 캠코로부터 제출받은 자료에 따르면 지난 8월 말 기준 캠코가 올해 저축은행권으로부터 사들인 무담보 채권액은 2786억원으로 집계됐다. 캠코는 금융회사가 보유한 취약가계 부실채권 등을 사들여 금융시장 건전성 제고를 지원한다.이 가운데 저축은행으로부터 인수한 무담보채권액은 2020년 430억원에서 2021년 670억원, 작년 2018억원으로 빠르게 늘고 있다. 올해 8월까지 집계된 규모만해도 벌써 지난해 인수액을 뛰어넘은 상태다.전체 무담보 채권 인수 규모가 코로나19 사태가 본격화된 2021년 폭증했다가 이후 안정세를 보이는 것과 다르다. 저축은행을 포함해 1금융권과 공공기관(주택금융공사, 주택도시보증공사, 신용보증기금 등) 등으로부터 매입한 캠코의 전체 무담보채권액은 2021년 5조원대로 급증했다가 작년 2조7773억원으로 감소한 상태다.이처럼 유독 저축은행으로부터 인수한 무담보채권액이 많아진건 취약차주들의 부실이 현실화되는 데 배경이 있다. 금리가 급격하게 오르고, 물가마저 뛰면서 다중채무자나 저신용자가 많은 2금융권 이용자들의 상환능력이 현저하게 떨어졌다는 얘기다.금융감독원에 따르면 상반기 저축은행 79곳의 연체율은 5.33%로 작년 말보다 1.92%포인트 늘었다.양정숙 의원은 “저축은행권 및 보험업, 여신전문업, 상호금융업, 대부업권에서 발생하는 부실채권이 급증하고 있다는 것은 그만큼 서민층의 경제 상황은 벼랑 끝에

100%|██████████| 36/36 [04:45<00:00,  7.94s/it]

10일 국회 정무위원회 소속 양정숙 의원실이 고금리 여파로 2금융권 차주가 늘어나면서 부실채권 규모가 급증한 것으로 나타났다며, 지난 8월 말 기준 캠코가 올해 저축은행권으로부터 사들인 무담보 채권액은 2786억원으로 집계됐고,이 가운데 저축은행으로부터 인수한 무담보채권액은 2020년 430억원에서 2021년 670억원, 작년 2018억원으로 빠르게 늘고 있다고 밝혔다.
 고금리·고물가·전기요금 인상 등 복합 요인…작년 2분기 엔데믹 기저효과도 자영업자 대출 연체액 7.3조 원으로 역대 최대 / 사진=연합뉴스올해 2분기 자영업자 가구가 이자 비용과 세금 등을 빼고 실제로 쓸 수 있는 돈이 20% 가까이 감소한 것으로 나타났습니다.고금리, 고물가, 전기요금 인상 등의 요인이 복합적으로 작용한 데다, 지난해 엔데믹(경제활동 재개)으로 소득이 많이 늘어난 기저효과도 있어서입니다.오늘(10일) 국회 산업통상자원중소벤처기업위원회 소속 더불어민주당 김회재 의원이 국회입법조사처에 의뢰해 통계청 가계동향조사 마이크로데이터를 분석한 결과 올해 2분기 고용원이 있는 자영업자 가구의 실질 처분가능소득은 월평균 537만 원으로 지난해 동기보다 19.5% 줄었습니다.고용원이 없는 자영업자 가구의 실질 처분가능소득도 343만 원으로 16.2% 감소했습니다.처분가능소득은 가구 소득에서 이자 비용과 세금 등 비소비지출을 뺀 소득으로 가구가 실제로 쓸 수 있는 돈을 뜻합니다.실질 처분가능소득은 처분가능소득에서 물가 상승 영향을 뺀 수치입니다. 자영업자 대출 1000조 돌파 / 사진=연합뉴스자영업자 가구의 실질 처분가능소득 감소 폭은 임금근로자 가구보다 훨씬 컸습니다.2분기 상용근로자 가구의 실질 처분가능소득은 월평균 430만 원으로 지난해 동기보다 0.6% 오히려 늘었습니다. 임시근로자(243만 원)는 6.6%, 일용근로자(252만 원)는 12.5% 각각 줄었습니다.더구나 자영업자 가구의 실질 처분가능소득 감소 폭은 갈수록 커지고 있습니다.고용원이 있는 자영업자 가구의 실질 처분가능소득 감

100%|██████████| 36/36 [04:52<00:00,  8.13s/it]

올해 2분기 고용원이 있는 자영업자 가구의 실질 처분가능소득은 월평균 430만 원으로 지난해 동기보다 0.6% 오히려 늘었고 실질 처분가능소득 감소 폭은 신종 코로나바이러스 감염증(코로나19) 사태 초기보다 훨씬 많아 고용원이 없는 자영업자 가구의 감소 폭은 지난해 3분기 2.6%에서 올해 2분기 16.2%로 커졌다.
                                                   제목  \
0                            [이윤희의 매크로 VIEW] "금리의 보복"   
1                   사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다   
2                   사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다   
3             3분기 금융지주 실적 전망…KB "양호" vs 신한·하나·우리 "부진"   
4                      컨트리가든 "디폴트" 현실화 하나…구조조정 자문사 선임   
5                 "올 초 8억 하던 아파트가 벌써 12억"…세입자들 "발 동동"   
6                    역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 "속탄다"   
7             전쟁나도 오르는 코스피 "안심은 일러…연말까진 상방보다 "하방" 열려"   
8                   "구리 공급량 감소 추이 심각하다"…청정에너지 전환 더뎌지나   
9           이스라엘-하마스 전쟁에도…장중 환율, 1340원 중후반대 약보합[외환분석]   
10             [5차 중동전쟁 위기]국내 증시 영향 제한적…바닥 확인은 지연될 수도   
11              [뉴스in뉴스] 미국이 불붙인 한국 대출금리…레고랜드 여파는 부채질   
12                   "수백만명에 재앙 닥쳤는데…" 美 증시 상승한 결정적 이유   
13                 `힐스테

In [10]:
df

,제목,내용,링크,요약
0,"[이윤희의 매크로 VIEW] ""금리의 보복""",연합뉴스 어쩌면 밀린 공과금 청구서나 군자의 복수 계획처럼 유예됐을 뿐 언젠가는 ...,https://n.news.naver.com/mnews/article/029/000...,글로벌 금융위기와 코로나19 사태를 지나며 세계 각국이 공중에 뿌려댄 돈의 대가는 ...
1,"사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다",게티이미지뱅크일상 속 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 주화가 급증...,https://n.news.naver.com/mnews/article/469/000...,10일 국회 기획재정위원회 소속 서영교 더불어민주당 의원이 한국은행에서 받은 자료에...
2,"사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다",게티이미지뱅크일상 속 동전 사용이 크게 줄면서 한국은행 금고에 쌓이는 주화가 급증...,https://n.news.naver.com/mnews/article/469/000...,10일 국회 기획재정위원회 소속 서영교 더불어민주당 의원이 한국은행에서 받은 자료에...
3,"3분기 금융지주 실적 전망…KB ""양호"" vs 신한·하나·우리 ""부진""","SK증권 “KB금융, 양호한 대출 성장 바탕으로 안정적 이자이익 기록”하나증권 “...",https://n.news.naver.com/mnews/article/123/000...,금융업계에 의하면 10일 4대 금융지주의 지배주주순이익은 약 4조3816억원으로 전...
4,"컨트리가든 ""디폴트"" 현실화 하나…구조조정 자문사 선임","中집값 급락에 보유자산 처분 막혀…자문사 선정, 17일까지 역외채권 이자 1500...",https://n.news.naver.com/mnews/article/008/000...,중국 최대 부동산 개발업체 컨트리가든(비구이위안)그룹이 보유 자산 처분이 여의치 않...
5,"""올 초 8억 하던 아파트가 벌써 12억""…세입자들 ""발 동동""","""전세 매물, 다 어디 갔나요""가을 이사철, 세입자들 '발 동동'전셋값 상승·매물...",https://n.news.naver.com/mnews/article/015/000...,부동산부동산정보광장에 따르면 서울 아파트 전세 비중은 2년 4개월 만에 가장 높은 ...
6,"역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 ""속탄다""","직방 ""응답자 55%는 '그래도 연내 추가 대출 필요' 답변""고금리 상황이 지속되...",https://n.news.naver.com/mnews/article/374/000...,시방이 지난달 14일부터 28일까지 조사한 결과에 따르면 지난해와 비교해 이자 부담...
7,"전쟁나도 오르는 코스피 ""안심은 일러…연말까진 상방보다 ""하방"" 열려""",'확전' 가능성 차단 및 추가 금리인상 일축에 반등고금리 지속으로 투심 악화 당분...,https://n.news.naver.com/mnews/article/421/000...,이스라엘과 팔레스타인 무장정파 하마스의 무력 충돌로 글로벌 경제에도 긴장감이 맴돌았...
8,"""구리 공급량 감소 추이 심각하다""…청정에너지 전환 더뎌지나",이 기사는 국내 최대 해외 투자정보 플랫폼 한경 글로벌마켓에 게재된 기사입니다. ...,https://n.news.naver.com/mnews/article/015/000...,파이낸셜타임스(FT)는 글로벌 광산업체들이 지난 5~6일 영국 런던에서 개최된 FT...
9,"이스라엘-하마스 전쟁에도…장중 환율, 1340원 중후반대 약보합[외환분석]",중동 전쟁에 안전자산 선호…국고채 10년 13bp 하락달러인덱스 장중 106선 하...,https://n.news.naver.com/mnews/article/018/000...,"이스국이 팔레스타인 무장 정파 하마스의 교전에 사실상 전쟁을 선포한 가운데, 전쟁이..."


No charts were generated by quickchart


In [19]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 데이터 로드 (예제: 데이터프레임 df)
# df = pd.read_csv('your_dataset.csv')

# BERT 모델 이름
model_name = "bert-base-multilingual-cased"

# 텍스트 감정 분석 함수 정의
def sentiment_analysis(text):
    # BERT 모델 및 토크나이저 로드 (각 반복마다 초기화)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # '제목'과 '내용' 열을 합쳐서 'text' 열 생성
    text_combined = text['제목'] + ' ' + text['내용']

    # 텍스트를 감정 분석 모델에 입력하여 예측
    inputs = tokenizer(text_combined, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # 예측된 클래스에 따라 감정 레이블을 반환
    if predicted_class == 0:
        return '부정'
    elif predicted_class == 1:
        return '중립'
    else:
        return '긍정'

# 'text' 컬럼에 대한 감정 분석 수행
df['감정'] = df.apply(sentiment_analysis, axis=1)

# 'text' 열 삭제
df.drop(columns=['text'], inplace=True)

# 결과 확인
print(df)

# 결과 데이터프레임을 CSV 파일로 저장
df.to_csv('output.csv', index=False)  # 'output.csv'는 저장할 파일명입니다.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkp

                                                   제목  \
0                            [이윤희의 매크로 VIEW] "금리의 보복"   
1                   사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다   
2                   사라지는 동전, 어디에 있나 했더니... 한은 금고에 쌓였다   
3             3분기 금융지주 실적 전망…KB "양호" vs 신한·하나·우리 "부진"   
4                      컨트리가든 "디폴트" 현실화 하나…구조조정 자문사 선임   
5                 "올 초 8억 하던 아파트가 벌써 12억"…세입자들 "발 동동"   
6                    역전세 한숨 돌렸더니, 집주인 이번엔 대출이자에 "속탄다"   
7             전쟁나도 오르는 코스피 "안심은 일러…연말까진 상방보다 "하방" 열려"   
8                   "구리 공급량 감소 추이 심각하다"…청정에너지 전환 더뎌지나   
9           이스라엘-하마스 전쟁에도…장중 환율, 1340원 중후반대 약보합[외환분석]   
10             [5차 중동전쟁 위기]국내 증시 영향 제한적…바닥 확인은 지연될 수도   
11              [뉴스in뉴스] 미국이 불붙인 한국 대출금리…레고랜드 여파는 부채질   
12                   "수백만명에 재앙 닥쳤는데…" 美 증시 상승한 결정적 이유   
13                 `힐스테이트 과천 디센트로` 오피스텔 선착순 계약 열기 뜨겁네   
14            이자·세금 내고 나니 쓸 돈이 없어…자영업자 실질 처분가능소득 20%↓   
15                  "금리 고점 아직"…은행 대기 자금 한 달 새 10조 "쑥"   
16                 中 공기업, 제주드림타